In [0]:
import configparser
import glob
import os
import pandas as pd
import subprocess
import sys
import tarfile 
from urllib.request import urlretrieve
import json
import tempfile
import tensorflow as tf

In [2]:
!git clone  https://github.com/HisakaKoji/bert-japanese.git

Cloning into 'bert-japanese'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 218 (delta 31), reused 26 (delta 12), pack-reused 168
Receiving objects: 100% (218/218), 282.85 KiB | 4.49 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [3]:
%cd /content/bert-japanese/

/content/bert-japanese


In [4]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 282.45 KiB | 3.77 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [5]:
%cd ..

/content


In [6]:
!pip install -q -r bert-japanese/requirements.txt

     |████████████████████████████████| 1.0MB 3.5MB/s 


In [7]:
%cd bert-japanese/notebook

/content/bert-japanese/notebook


In [0]:
sys.path.append("../src")
from utils import str_to_value

In [0]:
sys.path.append("../bert")
import modeling

In [10]:
CURDIR = os.getcwd()
CONFIGPATH = os.path.join(CURDIR, os.pardir, 'config.ini')
config = configparser.ConfigParser()
config.read(CONFIGPATH)

['/content/bert-japanese/notebook/../config.ini']

In [11]:
from google.colab import auth
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [12]:
!gsutil cp gs://hisaka/model/wiki-ja.model /content/bert-japanese/model/
!gsutil cp gs://hisaka/model/wiki-ja.vocab /content/bert-japanese/model/

Copying gs://hisaka/model/wiki-ja.model...
- [1 files][786.8 KiB/786.8 KiB]                                                
Operation completed over 1 objects/786.8 KiB.                                    
Copying gs://hisaka/model/wiki-ja.vocab...
- [1 files][581.7 KiB/581.7 KiB]                                                
Operation completed over 1 objects/581.7 KiB.                                    


In [22]:
!gsutil cp gs://hisaka/model/model.ckpt-1400000* /content/bert-japanese/model/

Copying gs://hisaka/model/model.ckpt-1400000.data-00000-of-00001...
Copying gs://hisaka/model/model.ckpt-1400000.index...
Copying gs://hisaka/model/model.ckpt-1400000.meta...
| [3 files][  1.3 GiB/  1.3 GiB]   20.0 MiB/s                                   
Operation completed over 3 objects/1.3 GiB.                                      


In [0]:
FILEURL = config['FINETUNING-DATA']['FILEURL']
FILEPATH = config['FINETUNING-DATA']['FILEPATH']
EXTRACTDIR = config['FINETUNING-DATA']['TEXTDIR']

In [0]:
PRETRAINED_MODEL_PATH = '/content/bert-japanese/model/model.ckpt-1400000'

In [17]:
bert_config_file = tempfile.NamedTemporaryFile(mode='w+t', encoding='utf-8', suffix='.json')
bert_config_file.write(json.dumps({k:str_to_value(v) for k,v in config['BERT-CONFIG'].items()}))
bert_config_file.seek(0)
bert_config_file_path = str(bert_config_file.name)
bert_config = modeling.BertConfig.from_json_file(bert_config_file.name)

In [0]:
!echo '政府への抗議活動が続く香港で１日午後、数千人規模のデモ隊が香港国際空港の周辺に集まり、ターミナルを包囲した。出入り口にバリケードを築くなどして空港利用者にも影響が出たが、約２時間後に警察が排除した。デモとの関係は不明だが、出発が数時間遅れた便もあり、空のダイヤに乱れが生じた。' > /tmp/input.txt

In [0]:
!python ../src/extract_features.py \
  --input_file=/tmp/test.csv \
  --output_file=./output.txt \
  --vocab_file=../model/wiki-ja.vocab \
  --model_file=../model/wiki-ja.model \
  --bert_config_file={bert_config_file.name} \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --layers=-1,-2,-3,-4 \
  --max_seq_length=128 \
  --batch_size=8



W0919 23:52:27.965666 139925391214464 deprecation_wrapper.py:119] From ../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Loaded a trained SentencePiece model.
I0919 23:52:28.749710 139925391214464 utils.py:141] NumExpr defaulting to 2 threads.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0919 23:52:29.035075 139925391214464 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O r

Let's have a look at the features of the last layer for the word "人間".

The 0-th token is always [CLS], and the 1st token of a sentence is [▁]. So the word comes in 3rd position.

In [0]:
import json

outputs = json.load(open('output.json'))
print(outputs['features'][3]['token'])

への


The last year is layer 0, the one before is layer -1, etc...
The embeddings are stored in the *values* entry.

In [0]:
embeddings = outputs['features'][3]['layers'][0]['values']
print(len(embeddings))

768
